In [ ]:
# NLP Lecture @ Strive School - 21st July 2021
# NER update

'''
Since today we are exploring the world of natural language processing, we’ll deepen in the Named Entity Recognition technique: this is just one of the mechanisms that NLP embodies. The recognition of named entities as the process of automatic identification of the entities present in a text and consequent classification into predefined categories such as "person", "organization", "position" is a quite common activity and expect for English, trained models with spaCy offer few labels that could be improved through training.

Following the case study of this morning, try to emulate it in order to label all the brands present in the provided datasets, choosing the one you prefer OR trying to label all them and to train the model to recognize new different entities. The result should be twofold: the final model should be able to recognize brands that it has already seen, but already new ones.
The brands proposed in the dataset concern fashion, cars and food.
In order to test the accuracy of the model, test it with sentences and brands the model has never seen.

Sample of the dataset
---------------------
- Cate Blanchett in Armani Privé. Rating: 8. Concludes as a rare butterfly, or from Rorschach's Test, or from computerized axial tomography.
- I liked everything, recommend it! Another quality Xiaomi product...
- What is the price of that Fiat 500XL?

Info:
- Feel free to change or arrange a new dataset
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)

'''

In [1]:
# STEP 0 - PRE REQUISITES

# python -m spacy download en_core_web_lg

# TBD: Import libraries
import spacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

nlp = spacy.load("en_core_web_lg")

# TBD: Load preferred model

with open("electronics.txt") as file:
    dataset = file.read()

# TBD: Load the dataset and test it as-is

doc = nlp(dataset)
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

Entities: [('US', 'GPE'), ('Huawei', 'ORG'), ('Samsung', 'ORG'), ('Samsung', 'ORG'), ('Samsung', 'ORG'), ('the past years', 'DATE'), ('Samsung', 'ORG'), ('Samsung', 'ORG'), ('VLC', 'ORG'), ('samsung', 'ORG'), ('Huawei', 'ORG'), ('Huawei', 'ORG'), ('USA', 'GPE'), ('p30', 'PRODUCT'), ('614', 'MONEY'), ('Huawei P30 Pro', 'ORG'), ('July', 'DATE'), ('AT&T', 'ORG'), ('Tmobile', 'ORG'), ('USA', 'GPE'), ('US', 'GPE'), ('Samsung', 'ORG'), ('Apple', 'ORG'), ('the last ten years', 'DATE'), ('three', 'CARDINAL'), ('Huawei', 'ORG'), ('English', 'LANGUAGE'), ('5 minutes', 'TIME'), ('second', 'ORDINAL'), ('Huawei', 'ORG'), ('iPhone', 'ORG'), ('Huawei', 'ORG'), ('iPhone', 'ORG'), ('Huawei', 'ORG'), ('Redmi', 'ORG'), ('third', 'ORDINAL'), ('Xiaomi', 'PRODUCT'), ('the Redmi 8', 'ORG'), ('the Redmi 9', 'ORG'), ('Xiaomi', 'PRODUCT'), ('second', 'ORDINAL'), ('second', 'ORDINAL'), ('second', 'ORDINAL'), ('XIAOMI', 'ORG'), ('MediaTek', 'ORG'), ('first', 'ORDINAL'), ('First', 'ORDINAL'), ('Xiaomi', 'PRODUCT')

In [9]:
# STEP 1 - TRAIN DATA

# Prepare training data

# TBD: define all the entities by extracting the words and their indexes from the dataset
# expected format is the following:  ("sentence", {"entities": [0,10, "FOOD"]})

words = ["apple", "samsung", "motorola", "xiaomi", "redmi", "nokia", "huawei", "android", "oppo"]

train_data = []

with open("electronics.txt") as file:
    dataset = file.readlines()
    for sentence in dataset:
        print("########")
        print("sentence: ", sentence )
        print("########")
        sentence = sentence.lower()
        entities = []
        for word in words:
            word = word.lower()
            if word in sentence:
                start_index = sentence.index(word)
                end_index = len(word) + start_index
                print("word: ", word)
                print("start index: ", start_index)
                print("end index: ", end_index)
                pos = (start_index, end_index, "ELECTRONICS BRAND")
                entities.append(pos)
        element = (sentence.rstrip('\n'), {"entities": entities})

        train_data.append(element)
        print("------------")
        print("element:", element)

########
sentence:  I am being forced to purchase a new device as all US networks are apparently no longer supporting Huawei devices in the very near future.

########
word:  huawei
start index:  98
end index:  104
------------
element: ('i am being forced to purchase a new device as all us networks are apparently no longer supporting huawei devices in the very near future.', {'entities': [(98, 104, 'ELECTRONICS BRAND')]})
########
sentence:  Samsung S20 is The best Phone that money can buy

########
word:  samsung
start index:  0
end index:  7
------------
element: ('samsung s20 is the best phone that money can buy', {'entities': [(0, 7, 'ELECTRONICS BRAND')]})
########
sentence:  I'm a Samsung fan, design and productivity is amazing and perfect, BUT it have a no-go for me: it poses a risk for phone owners.

########
word:  samsung
start index:  6
end index:  13
------------
element: ("i'm a samsung fan, design and productivity is amazing and perfect, but it have a no-go for me: it po

In [10]:
# STEP 2 - UPDATE MODEL

# TBD: load the needed pipeline

ner = nlp.get_pipe("ner")

# TBD: define the annotations

ner.add_label("ELECTRONICS BRAND")

# TBD: train the model

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# TBD: define the number of iterations, the batch size and the drop according to your experience or using an empirical value
# Train model
with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(30):
        print("Iteration #" + str(iteration))

        # Data shuffle for each iteration
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in spacy.util.minibatch(train_data, size=3):
            for text, annotations in batch:
                # Create an Example object
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=0.1)
        print("Losses:", losses)

# Save the model
# TBD:

output_dir = Path("/Users/franciscovarelacid/Desktop/Strive/final_strive_d2d/7. Natural Language Processing/6. NER/ner")
nlp.to_disk(output_dir)
print("Saved!")

Iteration #0
Losses: {'ner': 82.40579807433305}
Iteration #1
Losses: {'ner': 12.866254179494039}
Iteration #2
Losses: {'ner': 11.397734941136445}
Iteration #3
Losses: {'ner': 2.0350024840408865}
Iteration #4
Losses: {'ner': 2.7016529583044178}
Iteration #5
Losses: {'ner': 0.0004225143577061406}
Iteration #6
Losses: {'ner': 0.1263472969249236}
Iteration #7
Losses: {'ner': 8.812758064591546e-06}
Iteration #8
Losses: {'ner': 0.002768530530666307}
Iteration #9
Losses: {'ner': 4.8930046822955146e-08}
Iteration #10
Losses: {'ner': 1.0606677953161534e-06}
Iteration #11
Losses: {'ner': 1.2377236252961717e-07}
Iteration #12
Losses: {'ner': 1.6331706660068374e-07}
Iteration #13
Losses: {'ner': 1.822896998947705e-08}
Iteration #14
Losses: {'ner': 4.4061740119186255e-08}
Iteration #15
Losses: {'ner': 1.212458822996346e-07}
Iteration #16
Losses: {'ner': 7.974795343315012e-08}
Iteration #17
Losses: {'ner': 1.6993291613818655e-09}
Iteration #18
Losses: {'ner': 2.9155519419846765e-08}
Iteration #19
Lo

In [12]:
# STEP 3 - TEST THE UPDATED MODEL

# Load updated model
print("Loading model...")
nlp_updated = spacy.load(output_dir)
print("Model loaded.")

# TBD: test with a old sentence
doc = nlp_updated("I loved it very good, and my first xiaomi. Battery lasts all day excellent.")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# TBD: test with a new sentence and an old brand
doc = nlp_updated("Obviously my favorite brand is apple! Nothing beats them.")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# TBD: test with a new sentence and a new brand
doc = nlp_updated("Obviously my favorite brand is lg! Nothing beats them.")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

doc = nlp_updated("From all the phone brands in the world I would choose without a doubt Sony! Most reliable and durable.")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

Loading model...
Model loaded.
entities: [('xiaomi', 'ELECTRONICS BRAND')]
entities: [('apple', 'ELECTRONICS BRAND')]
entities: [('lg', 'ELECTRONICS BRAND')]
entities: [('Sony', 'ELECTRONICS BRAND')]


In [14]:
print("Loading model...")
nlp_updated = spacy.load(output_dir)
print("Model loaded.")

new_sentence = input("Input a new sentence with a electronics brand included: ")
doc = nlp_updated(new_sentence)
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

Loading model...
Model loaded.
entities: [('Sony', 'ELECTRONICS BRAND')]
